## LSA 실습

In [ ]:
import gensim
from gensim import corpora, models, matutils, utils
import pprint
import numpy as np
from pprint import pprint

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
import re
stopwords_en = stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


#### 코퍼스 정의

In [ ]:
documents = ["I want to watch a movie this weekend.",
             "I went shopping yesterday. New Zealand won the World Test Championship by beating India by eight wickets at Southampton.",
             "I don't watch cricket. Netflix and Amazon Prime have very good movies to watch.",
             "Movies are a nice way to chill however, this time I would like to paint and read some good books. It's been so long!",
             "This blueberry milkshake is so good! Try reading Dr. Joe Dispenza's books. His work is such a game-changer! His books helped to learn so much about how our thoughts impact our biology and how we can all rewire our brains."]

In [ ]:
def eng_preprocess(text):     # 영어 텍스트 전처리하는 과정을 함수로 만듦
    text = text.lower() # 대소문자 소문자로 통합(정규화)
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(t) for t in tokens] # 토큰화
    tokens = [t for t in tokens if t not in stopwords_en] # 불용어 제거
    tokens = [t for t in tokens if re.fullmatch('[a-z_-]+', t)]
    return tokens

# 문서 전처리하기
doc_tokenized = [eng_preprocess(doc) for doc in documents]
print("두 번째 문서의 토큰 수:", len(doc_tokenized[1]))

dictionary = corpora.Dictionary(doc_tokenized)
BoW_corpus = [dictionary.doc2bow(doc) for doc in doc_tokenized]

print("단어집(vocabulary) 단어 수:", len(dictionary))
print("문서 수:", len(BoW_corpus))

두 번째 문서의 토큰 수: 13
단어집(vocabulary) 단어 수: 49
문서 수: 5


#### LSA 모델 적용 - TF-IDF 행렬
* 코퍼스를 TF-IDF 행렬을 기반으로 셍성

In [ ]:
tfidf_model = models.TfidfModel(BoW_corpus, smartirs='nfn')    # n: tf 변형 안함, f: idf 사용, n: 문서 정규화 안함
tfidf_corpus = tfidf_model[BoW_corpus]

In [ ]:
tfidf_matrix = matutils.corpus2dense(tfidf_corpus, num_terms=len(dictionary)).transpose()
print("watch 단어의 인덱스는", dictionary.token2id['watch'])
print("세 번째 문서의 watch 단어의 tfidf값:", tfidf_matrix[2,2]) # [2(문서인덱스),2(단어인덱스)]

watch 단어의 인덱스는 2
세 번째 문서의 watch 단어의 tfidf값: 2.6438563


* TF-IDF 행렬을 토픽 수가 2개인 LSA 모델에 적용

In [ ]:
# LSA(LSI) 모델 적용
lsi_model = gensim.models.lsimodel.LsiModel(corpus=tfidf_corpus, id2word=dictionary, num_topics=2, random_seed=123)
# random_seed가 같으면 다시 실행해도 같은 결과를 얻음

* 토픽별 단어 할당값

In [ ]:
# 토픽별 단어 할당값 행렬
topic_word_assignment_mat = lsi_model.get_topics()
print("토픽별 단어 할당값의 행렬 표현:")
pprint(np.round(topic_word_assignment_mat, 3))

print("토픽별 중요한 단어 순서대로 표현:")
pprint(lsi_model.print_topics())

토픽별 단어 할당값의 행렬 표현:
array([[-0.009, -0.   , -0.003, -0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   , -0.002, -0.002, -0.084, -0.002, -0.002, -0.285, -0.026,
        -0.026, -0.026, -0.026, -0.026, -0.026, -0.026, -0.026, -0.026,
        -0.026, -0.238, -0.238, -0.238, -0.238, -0.238, -0.238, -0.238,
        -0.238, -0.238, -0.238, -0.238, -0.238, -0.238, -0.238, -0.238,
        -0.238],
       [ 0.   , -0.   , -0.   , -0.   ,  0.277,  0.277,  0.277,  0.277,
         0.277,  0.277,  0.277,  0.277,  0.277,  0.277,  0.277,  0.277,
         0.277, -0.   , -0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ]])
토픽별 중요한 단어 순서대로 표현:
[(0,
  '-0.285*"book" + -0.238*"biology" + -0.238*"thought" + 

In [ ]:
print(np.abs(topic_word_assignment_mat[0,:]))
# 가장 큰 값의 인덱스 도출
print("가장 큰 값을 가진 인덱스:",np.argmax(np.abs(topic_word_assignment_mat[0,:])))

[9.02417693e-03 2.93780515e-04 2.95661759e-03 2.93780515e-04
 3.02180491e-15 2.99682830e-15 2.97592109e-15 2.98371088e-15
 3.00212886e-15 3.00671476e-15 2.95203171e-15 2.98610208e-15
 2.98457253e-15 2.98623415e-15 2.98061219e-15 2.98264106e-15
 3.00874144e-15 2.44971597e-03 2.44971597e-03 8.44045098e-02
 2.44971597e-03 2.44971597e-03 2.85385783e-01 2.56886177e-02
 2.56886177e-02 2.56886177e-02 2.56886177e-02 2.56886177e-02
 2.56886177e-02 2.56886177e-02 2.56886177e-02 2.56886177e-02
 2.56886177e-02 2.37791588e-01 2.37791588e-01 2.37791588e-01
 2.37791588e-01 2.37791588e-01 2.37791588e-01 2.37791588e-01
 2.37791588e-01 2.37791588e-01 2.37791588e-01 2.37791588e-01
 2.37791588e-01 2.37791588e-01 2.37791588e-01 2.37791588e-01
 2.37791588e-01]
가장 큰 값을 가진 인덱스: 22


In [ ]:
print("첫번째 토픽의 가장 중요한 단어:", np.argmax(np.abs(topic_word_assignment_mat[0,:])))
print("인덱스 22의 단어는", dictionary[22])
print("첫번째 토픽의 인덱스 22 단어의 할당값은", topic_word_assignment_mat[0,22])
# 토픽의 이름 만들기
print("첫번째 토픽의 가장 영향력이 큰 단어가 book 이므로 <책>으로 볼 수 있다. 다른 영향력이 큰 단어들도 고려하여 다르게 지을 수도 있다.")

첫번째 토픽의 가장 중요한 단어: 22
인덱스 22의 단어는 book
첫번째 토픽의 인덱스 22 단어의 할당값은 -0.2853857828486046
첫번째 토픽의 가장 영향력이 큰 단어가 book 이므로 <책>으로 볼 수 있다. 다른 영향력이 큰 단어들도 고려하여 다르게 지을 수도 있다.


In [ ]:
# watch라는 단어의 인덱스 탐색
dictionary.token2id['watch']

2

In [ ]:
topic_word_assignment_mat[1,2] #[1(토픽),2(워치의 인덱스)]

-2.758966824486576e-16

In [ ]:
print("두번째 토픽의 watch 단어 할당값:", topic_word_assignment_mat[1,dictionary.token2id['watch']])

두번째 토픽의 watch 단어 할당값: -2.758966824486576e-16


* 문서별 토픽 할당값

In [ ]:
# 문서별 토픽 할당값
doc_topic_assignment = lsi_model[BoW_corpus]
print("문서별 토픽 할당값의 간단한 표현:")
pprint(list(doc_topic_assignment))

# 문서별 토픽 할당값 행렬 표현
doc_topic_assignment_mat = np.zeros(shape=(len(doc_topic_assignment), lsi_model.num_topics), dtype=float)
for doc_idx, doc_topics in enumerate(doc_topic_assignment):
    for topic_idx, topic_weight in doc_topics:
        doc_topic_assignment_mat[doc_idx, topic_idx] = topic_weight

print("문서별 토픽 할당값의 행렬 표현:")
pprint(np.round(doc_topic_assignment_mat, 3))
## 첫번째 문서에서는 첫번째 토픽의 영향력이 크다.

문서별 토픽 할당값의 간단한 표현:
[[(0, -0.012568355551302028)],
 [(1, 3.605551275463989)],
 [(0, -0.10914078579491655)],
 [(0, -0.6357006470953545)],
 [(0, -4.459841481984703)]]
문서별 토픽 할당값의 행렬 표현:
array([[-0.013,  0.   ],
       [ 0.   ,  3.606],
       [-0.109,  0.   ],
       [-0.636,  0.   ],
       [-4.46 ,  0.   ]])


In [ ]:
print("첫번째 문서의 토픽 할당값", doc_topic_assignment_mat[0,:])
print("절댓값이 첫번째 토픽이 더 크므로 첫번째 토픽의 영향력이 큼. 두번째 토픽의 영향력은 거의 0임.")
print("두번째 문서의 두번째 토픽 할당값", doc_topic_assignment_mat[1,1]) # [1(두번째 문서),1(두번째 토픽)]

첫번째 문서의 토픽 할당값 [-0.01256836  0.        ]
절댓값이 첫번째 토픽이 더 크므로 첫번째 토픽의 영향력이 큼. 두번째 토픽의 영향력은 거의 0임.
두번째 문서의 두번째 토픽 할당값 3.605551275463989


* 토픽별 코퍼스 내 중요도 정보

In [ ]:
singular_values = lsi_model.projection.s
print("토픽별 코퍼스 내 중요도(특이값):", singular_values)

토픽별 코퍼스 내 중요도(특이값): [9.70754999 8.3718308 ]


In [ ]:
print("두 번째 토픽의 특이값:", singular_values[1])

두 번째 토픽의 특이값: 8.371830804056806
